In [1]:
import numpy as np 
import torch
import torch.nn as nn
import pandas as pd
from torch import optim
from torch.utils.data import DataLoader, Dataset
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA


In [2]:
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()

        # define: encoder
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 16, 3, 3,1),
            nn.ReLU(),
            nn.MaxPool2d(2),

        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(16, 8, 3, 2,1),
            nn.ReLU(),
        )

        # define: decoder
        self.decoder1 = nn.Sequential(
            nn.ConvTranspose2d(8, 16, 3, 1),
            nn.ReLU()
        )
        self.decoder2 = nn.Sequential(
            nn.ConvTranspose2d(16, 8, 3, 2),
            nn.ReLU()
        )
        self.decoder3 = nn.Sequential(
            nn.ConvTranspose2d(8, 3, 2, 3),
            nn.Tanh(),
        )


    def forward(self, x):
        #print("x_shape:",x.shape)
        featuremap1 = self.conv1(x)
        #print("feature map shape:",featuremap1.shape)
        featuremap2 = self.conv2(featuremap1)
        #print("feature map2 shape:",featuremap2.shape)
        rec1 = self.decoder1(featuremap2)
        #print("rec1 shape:",rec1.shape)
        rec2 = self.decoder2(rec1)
        #print("rec2 shape:",rec2.shape)
        decoded = self.decoder3(rec2)
        #print("decoded shape:",decoded.shape)

        # Total AE: return latent & reconstruct
        return featuremap2, decoded

In [3]:
# detect is gpu available.
use_gpu = torch.cuda.is_available()

autoencoder = Autoencoder()

# load data and normalize to [-1, 1]
trainX = np.load('./trainX.npy')
trainX = np.transpose(trainX, (0, 3, 1, 2)) / 255. * 2 - 1
trainX = torch.Tensor(trainX)

# if use_gpu, send model / data to GPU.
if use_gpu:
    autoencoder.cuda()
    trainX = trainX.cuda()

# Dataloader: train shuffle = True
train_dataloader = DataLoader(trainX, batch_size=32, shuffle=True)
test_dataloader = DataLoader(trainX, batch_size=32, shuffle=False)



In [8]:
# We set criterion : L1 loss (or Mean Absolute Error, MAE)
criterion = nn.MSELoss()
optimizer = optim.Adam(autoencoder.parameters(), lr=0.001)

# Now, we train 20 epochs.
for epoch in range(30):

    cumulate_loss = 0
    for x in train_dataloader:

        latent, reconstruct = autoencoder(x)
        loss = criterion(reconstruct, x)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        cumulate_loss = loss.item() * x.shape[0]

    print(f'Epoch { "%03d" % epoch }: Loss : { "%.8f" % (cumulate_loss / trainX.shape[0])}')






Epoch 000: Loss : 0.00011309
Epoch 001: Loss : 0.00014832
Epoch 002: Loss : 0.00015307
Epoch 003: Loss : 0.00010022
Epoch 004: Loss : 0.00011603
Epoch 005: Loss : 0.00012703
Epoch 006: Loss : 0.00008642
Epoch 007: Loss : 0.00017353
Epoch 008: Loss : 0.00012527
Epoch 009: Loss : 0.00011759
Epoch 010: Loss : 0.00012355
Epoch 011: Loss : 0.00012690
Epoch 012: Loss : 0.00014929
Epoch 013: Loss : 0.00012021
Epoch 014: Loss : 0.00011258
Epoch 015: Loss : 0.00012217
Epoch 016: Loss : 0.00013346
Epoch 017: Loss : 0.00015293
Epoch 018: Loss : 0.00010646
Epoch 019: Loss : 0.00016174
Epoch 020: Loss : 0.00015675
Epoch 021: Loss : 0.00010934
Epoch 022: Loss : 0.00015357
Epoch 023: Loss : 0.00014601
Epoch 024: Loss : 0.00013330
Epoch 025: Loss : 0.00015498
Epoch 026: Loss : 0.00013863
Epoch 027: Loss : 0.00010758
Epoch 028: Loss : 0.00017182
Epoch 029: Loss : 0.00013061


In [9]:
# Collect the latents and stdardize it.
latents = []
reconstructs = []
for x in test_dataloader:
    latent, reconstruct = autoencoder(x)
    latents.append(latent.cpu().detach().numpy())
    reconstructs.append(reconstruct.cpu().detach().numpy())

latents = np.concatenate(latents, axis=0).reshape([9000, -1])
latents = (latents - np.mean(latents, axis=0)) / np.std(latents, axis=0)
print(latents.shape)

# Use PCA to lower dim of latents and use K-means to clustering.
latents = PCA(n_components=8).fit_transform(latents)
result = KMeans(n_clusters = 2).fit(latents).labels_

# We know first 5 labels are zeros, it's a mechanism to check are your answers
# need to be flipped or not.
if np.sum(result[:5]) >= 3:
    result = 1 - result
""""
if np.sum(result[:5]) != 0 or np.sum(result[:5])!=5:
    print("redo")
"""
# Generate your submission
df = pd.DataFrame({'id': np.arange(0,len(result)), 'label': result})
df.to_csv('change.csv',index=False)

(9000, 72)
